In [1]:
from dependencies import *

chats = pd.read_csv('chat_info.csv')
checked_chats = pd.read_csv('checked.csv')
errored = []
found = []
sessions = ["my_account"]

#forward:4273, 6092, 11995

In [3]:
async def chat_info(username, session):
    app = Client(session)
    async with Client(session) as app:
        try:
            chat_info = await app.get_chat(username)
            member_count = chat_info.members_count
            message_count= await app.get_chat_history_count(username)
            chat = {
                'ch_id': chat_info.id,
                'ch_username': chat_info.username,
                'ch_title': chat_info.title,
                'ch_type': chat_info.type,
                "mem_count": member_count,
                "mes_count": message_count}

            print(f"chat_info: info for {username} collected")
            return chat
        
        except Exception as e:
            print(f"chat_info: {e}: {username}")
            return None

In [4]:
async def first_time(username, session):

    async with Client(session) as app:
        try:
            async for message in app.get_chat_history(username, offset_id=5):
                if message.date is not None:
                    first_time = message.date.strftime("%Y.%m.%d")
            
            if first_time is None:
                print(f"first_time: first date is not found {username}")
                return None
            
            first_time = {"first_time": first_time}
            print(f"first_time: first date for {username} collected")
            return first_time
        except Exception as e:
            print(f"first_time: {e}: {username}")
            return None

In [5]:
async def messages(id, session):
    messages = []
    async with Client(session) as app:
        try:
            async for message in app.search_messages(id, limit=50):
                if message.text is not None:
                    messages.append(message.text)
            return messages
    
        except Exception as e:
            print(f"messages: {e}: {id}")
            return None

In [6]:
def detect_lang(messages):
    lang_list = []
    for string in messages:
        try:
            lang_list.append(detect(string))
        except Exception as e:
            print(f"detect_lang : Error detecting language: {e}")
            return None
        
    if not lang_list:
        # No language detected for any message
        return None

    lang_dict = {}
    for lang in lang_list:
        lang_dict[lang] = lang_dict.get(lang, 0) + 1
        
    dominant_lang = max(lang_dict, key=lang_dict.get)
    return dominant_lang

In [7]:
from dependencies import *

async def combined(username):

    chat_info_result = None
    first_time_result = None
    messages_result = None
    
    #tryng sessions one by one until one of them works. If none of them works, return none
    
    for session in sessions:
        chat_info_result = await chat_info(username, session)
        
        if chat_info_result is not None:
            break

    for session in sessions:
        first_time_result = await first_time(username, session)
        if first_time_result is not None:
            break
        
    for session in sessions:
        messages_result = await messages(username,session)
        if messages_result is not None:
            break
    
    #if one of them is none, return none
    if chat_info_result is None or first_time_result is None or messages_result is None:
        return None

    lang = detect_lang(messages_result)
    chat_info_result.update(first_time_result)
    chat_info_result.update({"lang": lang})
    
    return chat_info_result

In [8]:
async def conditions(username):
    
    chat = await combined(username)
    if chat is {} or chat is None:
        print("conditions: chat is empty")
        return None
    username = chat['ch_username']
    cr_time = datetime.strptime(chat['first_time'], '%Y.%m.%d')
    threshold = datetime.strptime('2017.12.30', '%Y.%m.%d')
    lang_list = ['ru', 'de', 'en', 'fr', 'ar']

    if cr_time > threshold:
        print(f"conditions: cr_time is later than threshold {username}")
        return None
    if chat["lang"] not in lang_list:
        print(f"conditions: lang is not in lang_list {username}")
        return None
    
    print(f"conditions: conditions are met {username}")
    return chat


In [9]:
async def checking(ch_ids):
    global checked_chats
    global chats
    
    for ch_id in ch_ids:
        index = ch_ids.index(ch_id)
        print(f"its index is {index}")
        if ch_id in checked_chats['ch_id'].tolist() or ch_id in chats['ch_id'].tolist():
            print(f"chat{ch_id} is already checked")
            continue
        chat_checked = await conditions(ch_id)
        if chat_checked is None:
            errored.append(ch_id)
            chat_checked = pd.DataFrame([ch_id], columns=['ch_id'])
            chat_checked.to_csv('checked.csv', mode='a', header=False, index=False)
            continue
        
        else:
            print(f"chat{ch_id} meets the conditions")
            chat_checked = pd.DataFrame(chat_checked, index=[index])
            chat_checked.to_csv('chat_info.csv', index=False, mode='a', header=False)
            found.append(ch_id)
    

    return found, errored

In [10]:
import pandas as pd
forward = pd.read_csv('forward.csv')
forward = forward['ch_id'].tolist()

In [ ]:
nest_asyncio.apply()
results = asyncio.run(checking(forward[11455:]))

In [84]:
#drop duplicates
chats = pd.read_csv('chat_info.csv')
chats = chats.drop_duplicates(subset=['ch_id'])
chats.to_csv('chat_info.csv', index=False)

checked_chats = pd.read_csv('checked.csv')
checked_chats = checked_chats.drop_duplicates(subset=['ch_id'])
checked_chats.to_csv('checked.csv', index=False)